# Introduction 🎢

This notebook will be mainly used for the capstone project.

# Coding Fun 🏖️💻

_Let's import the usual suspects_ 🧙‍♀️🧛🏻‍♂️🧟‍♂️ 🤓:

In [32]:
import pandas as pd
import numpy as np
import time # simple timer

_... and some new friends_  🦕🐢🐙🐿️:

In [33]:
import geocoder
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if not installed on your system
from geopy.geocoders import Nominatim # convert an address into latitude and l ongitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas d ataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if not installed on your system
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [34]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


---

# 3️⃣

# Segmenting and Clustering Neighborhoods in Toronto

In [35]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)[0]
df.shape

(287, 3)

In [36]:
df.head(3)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


Get rid off 🔫 that stupid `Not assigned` and fill it with `NaN`:

In [37]:
df = df.replace("Not assigned", np.nan)

##### a) Remove data without Bouroughs
>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [38]:
df = df[~df["Borough"].isna()]
df.shape

(210, 3)

#### b) Clean Neigbourhood Column
>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

_This stupid task took me almost 1 day!_ 😤 _to figure out how - trying different approaches!!! Nothing really worked out._ 😠 💣🔪🔫 _Now, I hate Pandas!_ 🐼

In [39]:
# using pandas built-in aggregate function did the trick
df = df.groupby('Postcode').agg(Bourough=("Borough","first"), 
                                 Neighbourhood=('Neighbourhood', list)).reset_index()
df.shape

(103, 3)

In [40]:
df.head()

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


#### c) Replace not assigned Neigbourhoods with Bouroughs
> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [41]:
#replace na in Neigbourhood with value in Bourough
df['Neighbourhood'].fillna(value=df['Bourough'],inplace=True)

In [42]:
df.head(5)

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


---

In [43]:
df.shape

(103, 3)

In [44]:
# Save DataFrame as file 
df.to_pickle("df_cleaned.pkl")

## Adding geo coordinates for Neighbourhoods

_First of all, I read the saved DataFrame:_ 📖👓

In [45]:
df = pd.read_pickle("df_cleaned.pkl")

_Okay. I admit: I'm lazy._ 🛌 _So I use the prepared table with the coordinates._

In [47]:
# I downloaded table from "https://cocl.us/Geospatial_data" as csv
coord = pd.read_csv("data/Geospatial_Coordinates.csv")
coord.shape

(103, 3)

In [48]:
# rename column "Postal Code" -> "Postcode" for merging
coord.rename(columns={"Postal Code": "Postcode"}, inplace=True)

In [49]:
coord.head(2)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


Merge both Dataframes

In [50]:
df = pd.merge(df, coord, how='inner', on='Postcode')

In [51]:
df.head(5)

,Postcode,Bourough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


--- 

## Explore and cluster the neighborhoods in Toronto

>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.   Just make sure:   
>- to add enough Markdown cells to explain what you decided to do and to report any observations you make.   
>- to generate maps to visualize your neighborhoods and how they cluster together.

## Download Foursquare Data for Toronto

In order to use Foursquares REST API you need to open a (free) account. After this you get an  Read my Login Information for Foursquare



In [52]:
# Read my Login Information for Foursquare
file = "data/my_secret_foursquare_login_data.json"
with open (file) as f:
    data = f.read()
login_data = json.loads(data)

In [53]:
CLIENT_ID = login_data["CLIENT_ID"]
CLIENT_SECRET = login_data["CLIENT_SECRET"]
VERSION = login_data["VERSION"]

In [148]:
# extracting the average coordinates for the toronto map
long = df.Longitude.mean()
lat = df.Latitude.mean()

_Where is Toronto?_ (Of course, in Canada 🇨🇦, but where exatly? Sorry 🙇🏻‍♂️ Canadians for my ignorance)

In [55]:
 # define a world map centered around Toronto with a low zoom level
toronto_map = folium.Map(location=[lat, long], zoom_start=2)
folium.features.CircleMarker(
            [lat, long],
            radius=10, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1).add_to(toronto_map)
# display world map
toronto_map

_Oh - I didn't know it is so near to the US boarder. Maybe this could be interesting for further investigations ..._ 🤔

## Download Venues Data for every Postcode from Foursquare

In [68]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=500, LIMIT=200):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:
# get venues by calling our function
start_timer = time.time() #time the scraping

venues = getNearbyVenues(postcodes=df['Postcode'],
                         latitudes=df['Latitude'],
                         longitudes=df['Longitude']
                        )

stop_timer = time.time()

print("Finished! It took {}s to get the venues.".format(round( (stop_timer-start_timer) ,1) ))

Finished! It took 95.6s to get the venues.


In [150]:
venues.shape

(2225, 7)

In [70]:
venues.head(3)

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [78]:
# add boroughs to the dataset
dx = pd.merge(venues, df[["Postcode", "Bourough"]], how="inner", on="Postcode")
dx.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Bourough
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,Scarborough
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,Scarborough
2,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,Scarborough
3,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa,Scarborough
4,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,Scarborough


How many categories do we have?

In [151]:
print('There are {} uniques categories.'.format(len(dx['Venue Category'].unique())))

There are 267 uniques categories.


In [152]:
by_borough = dx.groupby("Bourough") # create group object

# How many Postcodes has every Borough?
print("Number of Postcodes per Borough:")
print("===============================")
by_borough.count()["Postcode"].sort_values(ascending=False)

Number of Postcodes per Borough:


Bourough
Downtown Toronto    1307
North York           244
West Toronto         171
East Toronto         127
Central Toronto      112
Scarborough           90
East York             76
Etobicoke             68
York                  17
Mississauga           13
Name: Postcode, dtype: int64

In [153]:
# How many venue types has every Borough?
print("How many venue types does every Borough have?")
print("=============================================")
#by_borough.count()["Venue Category"].sort_values()
by_borough["Venue Category"].nunique().sort_values(ascending=False)

How many venue types does every Borough have?


Bourough
Downtown Toronto    206
North York          108
West Toronto         83
East Toronto         67
Central Toronto      61
Scarborough          55
East York            47
Etobicoke            39
York                 15
Mississauga          10
Name: Venue Category, dtype: int64

---
_Let's do some more advanced stuff..._ 😱

In [154]:
# one hot encoding
dx_onehot = pd.get_dummies(dx[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dx_onehot['Bourough'] = dx['Bourough'] 

# move neighborhood column to the first column
fixed_columns = [dx_onehot.columns[-1]] + list(dx_onehot.columns[:-1])
dx_onehot = dx_onehot[fixed_columns]

dx_onehot.head(3)

,Bourough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's do some sanity checking ...

In [155]:
print("Shape: ", dx_onehot.shape)
print( dx.groupby("Bourough")["Postcode"].count().sort_values(ascending=False).head())
print( dx.groupby("Bourough")["Venue Category"].nunique().sort_values(ascending=False).head())

Shape:  (2225, 268)
Bourough
Downtown Toronto    1307
North York           244
West Toronto         171
East Toronto         127
Central Toronto      112
Name: Postcode, dtype: int64
Bourough
Downtown Toronto    206
North York          108
West Toronto         83
East Toronto         67
Central Toronto      61
Name: Venue Category, dtype: int64


_Okay, that looks good. Let's continue with Clustering_ 📊... _But first, lets do some more exploratory research:_

In [156]:
dx_grouped = dx_onehot.groupby('Bourough').mean().reset_index()
dx_grouped.head(2)

,Bourough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.017857,...,0.000000,0.008929,0.00000,0.0,0.008929,0.0,0.000000,0.00000,0.000000,0.008929
1,Downtown Toronto,0.0,0.000765,0.000765,0.000765,0.000765,0.00153,0.000765,0.000765,0.016067,...,0.002295,0.011477,0.00153,0.0,0.003826,0.0,0.006886,0.00153,0.000765,0.003826


#### Let's print each Borough along with the top 5 most common venues

In [157]:
num_top_venues = 5

for borough in dx_grouped['Bourough']:
    print("----"+borough+"----")
    temp = dx_grouped[dx_grouped['Bourough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    #print(temp)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Park  0.06
2  Sandwich Place  0.06
3            Café  0.05
4             Gym  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.05
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1  Italian Restaurant  0.05
2         Coffee Shop  0.05
3                Café  0.04
4             Brewery  0.04


----East York----
         venue  freq
0  Coffee Shop  0.05
1         Park  0.05
2  Pizza Place  0.05
3         Bank  0.04
4     Pharmacy  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.07
2     Coffee Shop  0.06
3             Gym  0.04
4   Grocery Store  0.04


----Mississauga----
                 venue  freq
0                Hotel  0.15
1         Intersection  0.15
2          Coffee Shop  0.15
3                  Gym  0.08
4  American

#### Let's put this new information into our *Toronto* dataframe

First, let's write a function to sort the venues in descending order.

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Bourough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Bourough'] = dx_grouped['Bourough']

for ind in np.arange(dx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Bourough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Gym,Sushi Restaurant,Dessert Shop,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Bakery,Japanese Restaurant,Hotel,Italian Restaurant,Bar,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Pizza Place,Park,Ice Cream Shop,Pub,Sandwich Place
3,East York,Pizza Place,Park,Coffee Shop,Sandwich Place,Pharmacy,Bank,Sporting Goods Shop,Burger Joint,Gym,Skating Rink
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Gym,Fast Food Restaurant,Pharmacy,Grocery Store,Beer Store,Bakery,Convenience Store


## Cluster Analysis

Run *k*-means to cluster the bouroughs into 5 clusters.

In [160]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = dx_grouped.drop('Bourough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 0, 1, 3, 4, 3, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [161]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = manhattan_data
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Bourough'), on='Bourough')

toronto_merged.head() # check the last columns!

,Postcode,Bourough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,4,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,4,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,4,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,4,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,4,Chinese Restaurant,Coffee Shop,Bakery,Breakfast Spot,Fast Food Restaurant,Pizza Place,Pharmacy,Skating Rink,Park,Grocery Store


Finally, let's visualize the resulting clusters

In [149]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Bourough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---
# 🔚 🙏🏼 🙋🏻‍♂️